# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

In [120]:
import os
#os.environ['OPENAI_API_KEY'] = 'YOUR_KEY'
#os.environ['LLM_MODEL'] = 'gpt-4o'
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [121]:
llm_model = os.environ['LLM_MODEL'] 

In [122]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [123]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [124]:
from langchain.indexes import VectorstoreIndexCreator

In [125]:
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI

embeddings = OpenAIEmbeddings()
llm = OpenAI(temperature=0)


In [126]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
       embedding=embeddings
    
).from_loaders([loader])

In [127]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [128]:
response = index.query(query, llm=llm) 


In [129]:
display(Markdown(response))



| Name | Description | Sun Protection Rating |
| --- | --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | Made of 100% polyester, UPF 50+ rating, wrinkle-resistant, front and back cape venting, two front bellows pockets | SPF 50+, blocks 98% of harmful UV rays |
| Men's Plaid Tropic Shirt, Short-Sleeve | Made of 52% polyester and 48% nylon, UPF 50+ rating, SunSmart technology, wrinkle-free, front and back cape venting, two front bellows pockets | SPF 50+, blocks 98% of harmful UV rays |
| Men's TropicVibe Shirt, Short-Sleeve | Made of 71% nylon and 29% polyester, UPF 50+ rating, front and back cape venting, two front bellows pockets | SPF 50+, blocks 98% of harmful UV rays |
| Sun Shield Shirt | Made of 78% nylon and 22% Lycra Xtra Life fiber, UPF 50+ rating, moisture-wicking, abrasion-resistant, fits over swimsuit | SPF 50+, blocks 98% of harmful UV rays |

## Step By Step

In [130]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path=file)

In [131]:
docs = loader.load()

In [132]:
docs[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [88]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [133]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [134]:
print(len(embed))

1536


In [135]:
print(embed[:5])

[-0.021964654326438904, 0.006758837960660458, -0.01824948936700821, -0.03923514857888222, -0.014007173478603363]


In [92]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [93]:
query = "Please suggest a shirt with sunblocking"

In [94]:
docs = db.similarity_search(query)

In [95]:
len(docs)

4

In [102]:
docs[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 255}, page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.')

In [103]:
retriever = db.as_retriever()

In [104]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)

In [105]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [106]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


In [107]:
display(Markdown(response))

Here is a table summarizing the shirts with sun protection:

| Name                               | Description                                                                                                           | Size & Fit                      | Fabric & Care                                                                 | Additional Features                                                                 |
|------------------------------------|-----------------------------------------------------------------------------------------------------------------------|---------------------------------|--------------------------------------------------------------------------------|-------------------------------------------------------------------------------------|
| Sun Shield Shirt                   | High-performance sun shirt with UPF 50+ protection, blocks 98% of harmful UV rays.                                    | Slightly Fitted: Softly shapes the body. Falls at hip. | 78% nylon, 22% Lycra Xtra Life fiber. Handwash, line dry.                          | Moisture-wicking, abrasion-resistant, fits over swimsuit, recommended by The Skin Cancer Foundation. |
| Men's Plaid Tropic Shirt, Short-Sleeve | Ultracomfortable shirt with UPF 50+ protection, originally designed for fishing, great for travel.                     | Not specified                   | 52% polyester, 48% nylon. Machine washable and dryable.                           | Wrinkle-free, quick-drying, front and back cape venting, two front bellows pockets.  |
| Men's TropicVibe Shirt, Short-Sleeve | Lightweight sun-protection shirt with UPF 50+, relaxed fit, ideal for hot weather.                                     | Traditional Fit: Relaxed through the chest, sleeve, and waist. | Shell: 71% Nylon, 29% Polyester. Lining: 100% Polyester knit mesh. Machine wash and dry. | Wrinkle-resistant, front and back cape venting, two front bellows pockets.           |
| Men's Tropical Plaid Short-Sleeve Shirt | Lightest hot-weather shirt with UPF 50+ protection, relaxed fit, made of 100% polyester.                               | Traditional Fit: Relaxed through the chest, sleeve, and waist. | 100% polyester. Wrinkle-resistant.                                                | Front and back cape venting, two front bellows pockets.                              |

Each shirt offers UPF 50+ sun protection, blocking 98% of the sun's harmful rays, and includes features like moisture-wicking, wrinkle resistance, and venting for comfort in hot weather.

In [115]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [116]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [138]:
response = qa_stuff.invoke(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [139]:
display(Markdown(response["result"]))

Here is a table summarizing the shirts with sun protection:

| Name                                   | Summary                                                                                           |
|----------------------------------------|---------------------------------------------------------------------------------------------------|
| Men's Tropical Plaid Short-Sleeve Shirt| A lightweight, wrinkle-resistant shirt with UPF 50+ sun protection, made of 100% polyester.       |
| Men's Plaid Tropic Shirt, Short-Sleeve | A comfortable, quick-drying shirt with UPF 50+ sun protection, made of 52% polyester and 48% nylon.|
| Men's TropicVibe Shirt, Short-Sleeve   | A lightweight, wrinkle-resistant shirt with UPF 50+ sun protection, made of 71% nylon and 29% polyester.|
| Sun Shield Shirt                       | A slightly fitted shirt with UPF 50+ sun protection, made of 78% nylon and 22% Lycra Xtra Life fiber. |

Each shirt provides the highest rated sun protection possible, blocking 98% of the sun's harmful rays.